## Importing libraries

In [1]:
!pip3 install --upgrade tensorflow-gpu
!pip3 install tensorflow-hub


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import torch

## Dataset Loading and EDA

In [3]:
amazon_data = pd.read_csv('amz_com-ecommerce_sample.csv', encoding='latin-1')
amazon_data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [4]:
flipkart_data = pd.read_csv('flipkart_com-ecommerce_sample.csv', encoding='latin-1')
flipkart_data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [5]:
amazon_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   uniq_id                  20000 non-null  object
 1   crawl_timestamp          20000 non-null  object
 2   product_url              20000 non-null  object
 3   product_name             20000 non-null  object
 4   product_category_tree    20000 non-null  object
 5   pid                      20000 non-null  object
 6   retail_price             20000 non-null  int64 
 7   discounted_price         20000 non-null  int64 
 8   image                    19997 non-null  object
 9   is_FK_Advantage_product  20000 non-null  bool  
 10  description              19998 non-null  object
 11  product_rating           20000 non-null  object
 12  overall_rating           20000 non-null  object
 13  brand                    14136 non-null  object
 14  product_specifications   19986 non-nul

In [6]:
flipkart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In [7]:
amazon_data.shape, flipkart_data.shape

((20000, 15), (20000, 15))

## Using Universal Sentence Encoder

* It is one of the most well performing sentence embedding techniques proposed by Google. It is a pre-trained model used for finding embeddings.
* For this we need to install tensorflow and tensorflow hub.
* The model is available to us via tensorflow hub.

In [8]:
#Loading the model. This model converts a Phrase into a group of 512 semantic elements in a vector.

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module is loaded",module_url)

module is loaded https://tfhub.dev/google/universal-sentence-encoder/4


## Proceeding towards finding 'Title Similiarity' for product matching usecase.

In [9]:
# Finding embeddings for product title for both dataset.

amazon_embedding = model(amazon_data.product_name)
flipkart_embedding = model(flipkart_data.product_name)

In [10]:
# Embeddings are in the form of tf tensor.

amazon_embedding

<tf.Tensor: shape=(20000, 512), dtype=float32, numpy=
array([[ 0.03533603, -0.01567936, -0.00048282, ...,  0.05367972,
        -0.03440214, -0.05876047],
       [-0.05570024,  0.01934514, -0.05690151, ..., -0.04803508,
         0.06880123, -0.01235523],
       [ 0.01868252, -0.01210253, -0.0146432 , ...,  0.01036256,
         0.0513172 , -0.02117123],
       ...,
       [ 0.05938387, -0.00295221, -0.03464825, ...,  0.03233175,
         0.00308145, -0.01879093],
       [ 0.0593839 , -0.00295219, -0.03464826, ...,  0.03233175,
         0.00308145, -0.01879095],
       [ 0.0593839 , -0.00295219, -0.03464826, ...,  0.03233175,
         0.00308145, -0.01879095]], dtype=float32)>

In [11]:
len(amazon_embedding)

20000

In [12]:
# Finding title similarity using numpy dot product.

similarity =np.inner(amazon_embedding, flipkart_embedding)

In [13]:
# Converting similarity to pytorch tensor.
similarity = torch.tensor(similarity)

In [14]:
# Storing the index of matched flipkart and amazon products.

similar_prod_indexes = []

for i in range(20000):
    max_score_idxs = torch.topk(similarity[i],k=5,largest=True,sorted=True).indices  # Selecting top 5.
    for idx in max_score_idxs:
        if idx != i:
            similar_prod_indexes.append(idx)
            break

In [15]:
# In the form of torch tensor.

similar_prod_indexes

[tensor(3),
 tensor(7),
 tensor(16632),
 tensor(0),
 tensor(12),
 tensor(15347),
 tensor(3),
 tensor(1),
 tensor(15708),
 tensor(3),
 tensor(16632),
 tensor(18),
 tensor(4),
 tensor(0),
 tensor(9924),
 tensor(0),
 tensor(7),
 tensor(18997),
 tensor(11),
 tensor(1),
 tensor(4),
 tensor(3),
 tensor(14640),
 tensor(1226),
 tensor(1586),
 tensor(434),
 tensor(5628),
 tensor(1119),
 tensor(11170),
 tensor(1601),
 tensor(9447),
 tensor(1346),
 tensor(33),
 tensor(32),
 tensor(11090),
 tensor(8349),
 tensor(18151),
 tensor(35),
 tensor(6092),
 tensor(13804),
 tensor(16095),
 tensor(15298),
 tensor(1265),
 tensor(51),
 tensor(45),
 tensor(44),
 tensor(65),
 tensor(52),
 tensor(52),
 tensor(58),
 tensor(44),
 tensor(43),
 tensor(48),
 tensor(44),
 tensor(7861),
 tensor(44),
 tensor(44),
 tensor(49),
 tensor(49),
 tensor(17816),
 tensor(44),
 tensor(14854),
 tensor(49),
 tensor(44),
 tensor(65),
 tensor(46),
 tensor(49),
 tensor(75),
 tensor(70),
 tensor(73),
 tensor(68),
 tensor(15619),
 tensor

In [16]:

similar_prod_indexes = [x.item() for x in similar_prod_indexes]
type(similar_prod_indexes)

list

In [17]:
len(similar_prod_indexes)

20000

In [18]:
amazon_data.product_name[501], flipkart_data.product_name[similar_prod_indexes[501]]

("Allen Solly Girl's A-line Dress", "Allen Solly Girl's Layered Dress")

In [19]:
# Extracting similar products details in a list.

flipkart_prod_data = []
amazon_prod_data = []

for idx, prod_name in enumerate(amazon_data.product_name):
    flipkart_prod_data.append(flipkart_data.iloc[similar_prod_indexes[idx]])
    amazon_prod_data.append(amazon_data.iloc[idx])

In [20]:
# Creating dataframe for similar products
amazon_prod_data = pd.DataFrame(amazon_prod_data)
flipkart_prod_data = pd.DataFrame(flipkart_prod_data)

In [21]:
# Selecting columns for comparison between amazon and flipkart produnts.

amazon_prod_data = amazon_prod_data[['product_name','retail_price','discounted_price']]
flipkart_prod_data = flipkart_prod_data[['product_name','retail_price','discounted_price']]

In [22]:
amazon_prod_data.head()

,product_name,retail_price,discounted_price
0,Alisha Solid Women's Cycling Shorts,982,438
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121
2,AW Bellies,991,551
3,Alisha Solid Women's Cycling Shorts,694,325
4,Sicons All Purpose Arnica Dog Shampoo,208,258


In [23]:
# As there are some null values in flipkart prices, I am not handling null values now.
flipkart_prod_data.head()

,product_name,retail_price,discounted_price
3,Alisha Solid Women's Cycling Shorts,699.0,267.0
7,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0
16632,DressBerry Bellies,1799.0,1259.0
0,Alisha Solid Women's Cycling Shorts,999.0,379.0
12,Sicons All Purpose Tea Tree Dog Shampoo,NaN,NaN


In [24]:
# Changing column names in both the similar dataframe.

flipkart_prod_data.columns = ['Product name in Flipkart','Retail Price in Flipkart','Discounted Price in Flipkart']
amazon_prod_data.columns = ['Product name in Amazon','Retail Price in Amazon','Discounted Price in Amazon']

In [25]:
flipkart_prod_data.shape

(20000, 3)

In [26]:
amazon_prod_data.shape

(20000, 3)

In [27]:
# Merging both the similar dataframe.

df_final = pd.concat([flipkart_prod_data.reset_index(drop=True),amazon_prod_data],axis=1)

In [28]:
df_final.head(20)

,Product name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart,Product name in Amazon,Retail Price in Amazon,Discounted Price in Amazon
0,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,982,438
1,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32143,29121
2,DressBerry Bellies,1799.0,1259.0,AW Bellies,991,551
3,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,694,325
4,Sicons All Purpose Tea Tree Dog Shampoo,NaN,NaN,Sicons All Purpose Arnica Dog Shampoo,208,258
5,"NEVI Elegant Swan Swarovski Crystal, Crystal A...",5900.0,2799.0,Eternal Gandhi Super Series Crystal Paper Weig...,427,473
6,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,1198,602
7,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32137,28664
8,Provogue Corporate Casuals,2399.0,2399.0,"dilli bazaaar Bellies, Corporate Casuals, Casuals",682,385
9,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,1197,542


In [29]:
df_final.shape

(20000, 6)